In [1]:
import numpy as np
import scipy.stats as stats
from math import log, sqrt, exp
import math
import pandas as pd
from scipy.stats import norm
import datetime
from scipy.optimize import brentq
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
import statsmodels.api as sm

## Problem 1

In [2]:
# Input parameters
S = 165  # Current stock price
K = 165  # Strike price
t = (np.datetime64('2022-03-13') - np.datetime64('2022-03-13')).astype(int) / 365.0
T = (np.datetime64('2022-04-15') - np.datetime64('2022-03-13')).astype(int) / 365.0
r = 0.0425  # Risk-free interest rate
q = 0.0053  # Continuously compounding dividend yield
sigma = 0.20  # Implied volatility

In [3]:
def gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call'):
    tau = T - t
    d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
    d2 = d1 - sigma * sqrt(tau)

    if option_type == 'call':
        delta = exp(-q * tau) * norm.pdf(d1)
        gamma = exp(-q * tau) * norm.pdf(d1) / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * norm.pdf(d1) * sigma / (2 * np.sqrt(tau)) - r * K * np.exp(-r * tau) * norm.cdf(-d2) + q * S * np.exp(-q * tau) * norm.cdf(-d1)
        rho = K * tau * np.exp(-r * tau) * norm.cdf(d2)
    elif option_type == 'put':
        delta = -exp(-q * tau) * (1 - norm.pdf(d1))
        gamma = exp(-q * tau) * norm.pdf(d1) / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * norm.pdf(d1) * sigma / (2 * np.sqrt(tau)) + r * K * np.exp(-r * tau) * norm.cdf(-d2) - q * S * np.exp(-q * tau) * norm.cdf(-d1)
        rho = -K * tau * np.exp(-r * tau) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return delta, gamma, vega, theta, rho

In [4]:
call_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call')
put_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks)
print("Put Greeks: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks)

Call Greeks: Delta, Gamma, Vega, Theta, Rho
(0.39727945117321767, 0.040037930803986446, 19.710179716477544, -24.835212485329674, 7.583586080244792)
Put Greeks: Delta, Gamma, Vega, Theta, Rho
(-0.6022414855320726, 0.040037930803986446, 19.710179716477544, -18.807899770342896, -7.277010958127815)


In [5]:
def finite_difference(S, K, t, T, r, q, sigma, option_type='call', delta_shift=0.01):
    S_up = S + S * delta_shift
    S_down = S - S * delta_shift
    sigma_up = sigma + delta_shift
    sigma_down = sigma - delta_shift
    r_up = r + delta_shift
    r_down = r - delta_shift

    def option_price(S, K, t, T, r, q, sigma, option_type):
        tau = T - t
        d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
        d2 = d1 - sigma * sqrt(tau)

        if option_type == 'call':
            return S * exp(-q * tau) * stats.norm.cdf(d1) - K * exp(-r * tau) * stats.norm.cdf(d2)
        elif option_type == 'put':
            return K * exp(-r * tau) * stats.norm.cdf(-d2) - S * exp(-q * tau) * stats.norm.cdf(-d1)

    # Finite difference calculations
    delta_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - option_price(S_down, K, t, T, r, q, sigma, option_type)) / (2 * S * delta_shift)
    gamma_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - 2 * option_price(S, K, t, T, r, q, sigma, option_type) + option_price(S_down, K, t, T, r, q, sigma, option_type)) / (S * delta_shift)**2
    vega_fd = (option_price(S, K, t, T, r, q, sigma_up, option_type) - option_price(S, K, t, T, r, q, sigma_down, option_type)) / (2 * 0.01)
    theta_fd = - (option_price(S, K, t + 1/365, T, r, q, sigma, option_type) - option_price(S, K, t, T, r, q, sigma, option_type)) / (-1/365)
    rho_fd = (option_price(S, K, t, T, r_up, q, sigma, option_type) - option_price(S, K, t, T, r_down, q, sigma, option_type)) / (2 * 0.01)

    return delta_fd, gamma_fd, vega_fd, theta_fd, rho_fd

In [6]:
call_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='call')
put_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks_fd)
print("Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks_fd)

Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(0.5337431087559965, 0.039948679750147466, 19.71009507625965, -25.06750653518303, 7.583554488655153)
Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(-0.4657778279492933, 0.03994867975014225, 19.71009507625965, -18.955580817535633, -7.277044574261282)


In [7]:
# Combine the results in a dictionary
greek_data = {
    "Call Closed-form": call_greeks,
    "Call Finite Diff.": call_greeks_fd,
    "Put Closed-form": put_greeks,
    "Put Finite Diff.": put_greeks_fd
}

# Create a DataFrame
greeks_comparison = pd.DataFrame(greek_data, index=["Delta", "Gamma", "Vega", "Theta", "Rho"])

# Display the DataFrame
print(greeks_comparison)

       Call Closed-form  Call Finite Diff.  Put Closed-form  Put Finite Diff.
Delta          0.397279           0.533743        -0.602241         -0.465778
Gamma          0.040038           0.039949         0.040038          0.039949
Vega          19.710180          19.710095        19.710180         19.710095
Theta        -24.835212         -25.067507       -18.807900        -18.955581
Rho            7.583586           7.583554        -7.277011         -7.277045


In [8]:
# Option parameters
S0 = 165      # Stock price
K = 165       # Strike price
T = 33/365    # Time to expiration (in years)
r = 0.0425    # Risk-free rate
q = 0.0053    # Dividend yield
sigma = 0.20  # Implied volatility
dividend_date = 26/365  # Dividend payment date
dividend = 0.88
n = 100
div_date = np.datetime64('2022-04-11')

# Binomial tree parameters
N = 1000    # Number of time steps
dt = T/N    # Time step size
u = math.exp(sigma*math.sqrt(dt))   # Up factor
d = 1/u                            # Down factor
p = (math.exp((r-q)*dt)-d)/(u-d)    # Probability of up move

# Compute dividend adjusted stock price
S_adjusted = S0 - math.exp(-q*T)*dividend

# Initialize the option values at expiration
call_values = [max(S_adjusted*(u**j)*(d**(N-j))-K, 0) for j in range(N+1)]
put_values = [max(K-S_adjusted*(u**j)*(d**(N-j)), 0) for j in range(N+1)]

# Move backwards through the tree
for i in range(N-1, -1, -1):
    for j in range(i+1):
        # Compute the expected value of the option at the next time step
        expected_value = math.exp(-r*dt)*(p*call_values[j+1]+(1-p)*call_values[j])
        # Check if early exercise is optimal for the call option
        exercise_value = S_adjusted*(u**j)*(d**(i-j))-K
        call_values[j] = max(expected_value, exercise_value)
        # Compute the expected value of the option at the next time step
        expected_value = math.exp(-r*dt)*(p*put_values[j+1]+(1-p)*put_values[j])
        # Check if early exercise is optimal for the put option
        exercise_value = K-S_adjusted*(u**j)*(d**(i-j))
        put_values[j] = max(expected_value, exercise_value)

# Compute the option values with dividend
call_with_dividend = call_values[0]
put_with_dividend = put_values[0]

# Compute the option values without dividend
call_without_dividend = call_values[0] + dividend*math.exp(-r*dividend_date)
put_without_dividend = put_values[0] - dividend*math.exp(-r*dividend_date)

print("Call with dividend:", round(call_with_dividend, 2))
print("Put with dividend:", round(put_with_dividend, 2))
print("Call without dividend:", round(call_without_dividend, 2))
print("Put without dividend:", round(put_without_dividend, 2))

Call with dividend: 3.78
Put with dividend: 4.15
Call without dividend: 4.66
Put without dividend: 3.27


In [9]:
def binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r) * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Create the stock price tree
    stock = np.zeros((n + 1, n + 1))
    stock[0, 0] = S

    for i in range(1, n + 1):
        stock[i, 0] = stock[i - 1, 0] * u
        for j in range(1, i + 1):
            stock[i, j] = stock[i - 1, j - 1] * d

    # Adjust stock prices for dividends
    if div_date is not None and exercise_type == 'american':
        div_steps = int((div_date - np.datetime64('2022-03-13')).astype(int) / 365.0 * n)
        stock[div_steps:] -= div_amt

    # Create the option price tree
    option = np.zeros_like(stock)

    # Calculate the option's value at maturity
    if option_type == 'call':
        option[:, -1] = np.maximum(stock[:, -1] - K, 0)
    elif option_type == 'put':
        option[:, -1] = np.maximum(K - stock[:, -1], 0)

    # Step back through the tree to calculate the option's value at each node
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option[i, j] = discount * (p * option[i + 1, j] + (1 - p) * option[i + 1, j + 1])

            # Calculate the option's value if exercised early
            if exercise_type == 'american':
                if option_type == 'call':
                    early_exercise = np.maximum(stock[i, j] - K, 0)
                elif option_type == 'put':
                    early_exercise = np.maximum(K - stock[i, j], 0)

                option[i, j] = np.maximum(option[i, j], early_exercise)

    return option

def binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0, delta_shift=0.01, rate_shift=0.0001, vol_shift=0.01):
    option_tree = binomial_tree(S, K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_up = binomial_tree(S * (1 + delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_down = binomial_tree(S * (1 - delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_up = binomial_tree(S, K, T, r, sigma + vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_down = binomial_tree(S, K, T, r, sigma - vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_up = binomial_tree(S, K, T, r + rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_down = binomial_tree(S, K, T, r - rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)

    # Calculate Greeks
    delta = (option_tree_S_up[0, 0] - option_tree_S_down[0, 0]) / (2 * S * delta_shift)
    gamma = (option_tree_S_up[0, 0] - 2 * option_tree[0, 0] + option_tree_S_down[0, 0])/ (S * delta_shift) ** 2
    theta = -(option_tree[0, 0] - binomial_tree(S, K, T - 1/n, r, sigma, n, option_type, exercise_type, div_date, div_amt)[0, 0]) / (1/n)

    vega = (option_tree_vol_up[0, 0] - option_tree_vol_down[0, 0]) / (2 * vol_shift)
    rho = (option_tree_rate_up[0, 0] - option_tree_rate_down[0, 0]) / (2 * rate_shift)

    return delta, gamma, theta, vega, rho

In [10]:
# calculate greeks
call_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american')
put_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american')
call_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=dividend)
put_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=dividend)

# create dataframe for greeks
greek_labels = ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']
call_greeks = pd.DataFrame({'Call without div': call_greeks_without_div, 'Call with div': call_greeks_with_div}, index=greek_labels)
put_greeks = pd.DataFrame({'Put without div': put_greeks_without_div, 'Put with div': put_greeks_with_div}, index=greek_labels)

# display greeks dataframes
print('Call Option Greeks:')
print(call_greeks)

print('\nPut Option Greeks:')
print(put_greeks)

Call Option Greeks:
       Call without div  Call with div
Delta          0.536898       0.500120
Gamma          0.038292       0.045113
Theta        -25.932238     -25.884008
Vega          19.657366      19.821456
Rho            7.550520       7.066599

Put Option Greeks:
       Put without div  Put with div
Delta        -0.471247     -0.508301
Gamma         0.039587      0.046210
Theta       -19.577328    -19.518492
Vega         19.621387     19.702896
Rho          -5.868226     -6.173442


In [11]:
# Given parameters
S0 = 165
K = 165
t = (np.datetime64('2022-04-15') - np.datetime64('2022-03-13')).astype('timedelta64[D]').astype(int) / 365
r = 0.0425
q = 0.0053
sigma = 0.20
dividend = 0.88

# Calculate the present value of the dividend
PV_dividend = dividend * np.exp(-q * (t - (np.datetime64('2022-04-11') - np.datetime64('2022-03-13')).astype('timedelta64[D]').astype(int) / 365))

# Calculate the d1 and d2 terms
d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
d2 = d1 - sigma * np.sqrt(t)

# Calculate the call and put option prices
Call = S0 * np.exp(-q * t) * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
Put = K * np.exp(-r * t) * norm.cdf(-d2) - S0 * np.exp(-q * t) * norm.cdf(-d1)

# Calculate the call and put option prices with the dividend
Call_dividend = (S0 - PV_dividend) * np.exp(-q * t) * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
Put_dividend = K * np.exp(-r * t) * norm.cdf(-d2) - (S0 - PV_dividend) * np.exp(-q * t) * norm.cdf(-d1)

# Calculate the sensitivity of the call and put to a change in dividend amount
Delta_call_dividend = (Call_dividend - Call) / dividend
Delta_put_dividend = (Put_dividend - Put) / dividend

print("Delta Call with Dividend: ", Delta_call_dividend)
print("Delta Put with Dividend: ", Delta_put_dividend)

Delta Call with Dividend:  -0.5339781069654976
Delta Put with Dividend:  0.4654847770590109


## Problem 2

In [12]:
import var as r

In [13]:
# read data from DailyPrices.csv
data = pd.read_csv('DailyPrices.csv')

# extract AAPL closing prices
aapl_prices = data['AAPL']

# compute AAPL returns
aapl_returns = np.log(aapl_prices / aapl_prices.shift(1))

# drop the first row (which will be NaN)
aapl_returns = aapl_returns.dropna()

# fit a normal distribution to AAPL returns
mu = 0.0 # assume zero mean return
sigma = np.std(aapl_returns)
aapl_dist = norm(mu, sigma)

# simulate 10 days' worth of returns
n_simulations = 1000
n_days = 10
simulated_returns = aapl_dist.rvs(size=(n_simulations, n_days))

# calculate simulated prices 10 days ahead
initial_price = 151.03
simulated_prices = initial_price * np.exp(np.cumsum(simulated_returns, axis=1))

# get the 10th day's results
simulation_price = simulated_prices[:, -1].tolist()

In [14]:
# Black-Scholes formula for European options
def black_scholes(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'Call':
        price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'Put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    return price

# Function to find the implied volatility
def implied_volatility(S, K, T, r, q, market_price, option_type):
    def objective_function(sigma):
        return black_scholes(S, K, T, r, q, sigma, option_type) - market_price

    try:
        result = brentq(objective_function, 1e-6, 1, full_output=False, disp=False)
    except ValueError:
        result = np.nan
    return result

In [15]:
def portfolio_value(portfolio, underlying_range, r, q, today, d_ahead = 0):
    values = []

    for underlying in underlying_range:
        total_value = 0

        for _, row in portfolio.iterrows():
            holding = row['Holding']

            if row['Type'] == 'Stock':
                total_value += holding * underlying
            elif row['Type'] == 'Option':
                T = ((row['ExpirationDate'] - today) / pd.Timedelta(1, 'D') - d_ahead) / 365
                option_value = black_scholes(underlying, row['Strike'], r, q, T, row['Implied_Volatility'], row['OptionType'])
                total_value += holding * option_value
        
        values.append(total_value)

    return values

In [16]:
# risk free rate and dividend
rf = 0.0425
q = 0.0053
S = 151.03
port = pd.read_csv('problem2.csv')
current_date = datetime.date(2023, 3, 3)
current_date = pd.to_datetime(current_date)
port['ExpirationDate'] = pd.to_datetime(port['ExpirationDate'])
port['T'] = (port['ExpirationDate'] - current_date).dt.days / 365
portfolios = port['Portfolio'].unique()

port['Implied_Volatility'] = port.apply(
    lambda row: implied_volatility(S, row['Strike'], rf, q, row['T'], row['CurrentPrice'], row['OptionType'])
    if row['Type'] == 'Option' else np.nan,
    axis=1
)
port.head()

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,T,Implied_Volatility
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.542910
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.134247,0.401306
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.542910
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.134247,0.401306
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.542910


In [17]:
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=portfolios, index=['Mean', 'VaR', 'ES'])

# Iterate through each portfolio and calculate the Mean, VaR, and ES
for portname in portfolios:
    portfolio_data = port[port['Portfolio'] == portname]
    simulate_portfolio_values = pd.DataFrame(portfolio_value(portfolio_data, simulation_price, rf, q, current_date, d_ahead=10))
    current_portfolio_values = pd.DataFrame(portfolio_value(portfolio_data, [initial_price], rf, q, current_date))

    # Calculate the difference between simulated and current portfolio values
    current_portfolio_value = current_portfolio_values.values[0][0]
    diff_portfolio_values = simulate_portfolio_values.apply(lambda x: x - current_portfolio_value)

    #print(diff_portfolio_values.values[0])
    # Calculate the Mean, VaR, and ES for the difference
    Mean = diff_portfolio_values.mean().values[0]

    var = r.calculate_var(diff_portfolio_values.values)
    es = r.calculate_es(diff_portfolio_values.values, var)

    # Add the results to the DataFrame
    results_df.at['Mean', portname] = Mean
    results_df.at['VaR', portname] = var
    results_df.at['ES', portname] = es

# Display the results
results_df.T

,Mean,VaR,ES
Straddle,2.982048,0.009344,0.018604
SynLong,0.710028,17.702116,21.874643
CallSpread,0.240339,3.525782,3.91628
PutSpread,0.258879,2.712724,2.857014
Stock,0.886034,17.001801,20.93669
Call,1.846038,5.590132,6.054338
Put,1.13601,4.426767,4.633052
CoveredCall,-0.924905,13.397191,17.13173
ProtectedPut,1.951894,7.31635,7.836915


In [18]:
# Create an empty DataFrame to store the results
results_dn_df = pd.DataFrame(columns=portfolios, index=['VaR', 'ES'])

def delta(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    if option_type == 'Call':
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'Put':
        delta = np.exp(-q * T) * (norm.cdf(d1) - 1)

    return delta

def delta_normal_var_pnl(portfolio, initial_price, rf, q, current_date, simulated_prices):
    portfolio_delta = 0

    for _, row in portfolio.iterrows():
        holding = row['Holding']

        if row['Type'] == 'Stock':
            portfolio_delta += holding
        elif row['Type'] == 'Option':
            T = ((row['ExpirationDate'] - current_date) / pd.Timedelta(1, 'D')) / 365
            sigma = row['Implied_Volatility']
            option_delta = delta(initial_price, row['Strike'], T, rf, q, sigma, row['OptionType'])
            portfolio_delta += holding * option_delta

    simulated_pnl = (simulated_prices - initial_price) * portfolio_delta
    dn_var = np.percentile(simulated_pnl, 5)
    return dn_var, simulated_pnl

# Calculate Delta-Normal VaR and ES for each portfolio
for portname in portfolios:
    portfolio_data = port[port['Portfolio'] == portname]
    dn_var, pnl = delta_normal_var_pnl(portfolio_data, initial_price, rf, q, current_date, simulated_prices[:, -1])
    dn_es = np.mean(pnl[pnl <= dn_var])

    results_dn_df.at['VaR', portname] = -dn_var
    results_dn_df.at['ES', portname] = -dn_es

# Display the results
results_dn_df.T

,VaR,ES
Straddle,2.112789,2.601771
SynLong,17.022811,20.962562
CallSpread,2.614241,3.21928
PutSpread,2.87778,3.718832
Stock,17.001801,20.93669
Call,9.5678,11.782167
Put,8.309372,10.737845
CoveredCall,8.687934,10.698666
ProtectedPut,10.956413,13.49216


## Problem 3

In [19]:
from datetime import datetime

In [20]:
# Load the datasets
fama_french = pd.read_csv('F-F_Research_Data_Factors_daily.csv')
momentum = pd.read_csv('F-F_Momentum_Factor_daily.csv')
prices = pd.read_csv('DailyPrices.csv')


# Filter the datasets for the past 10 years
current_year = datetime.now().year
start_date = (current_year - 10) * 10000 + 101
end_date = (current_year - 1) * 10000 + 1231

fama_french = fama_french[(fama_french['Date'] >= start_date) & (fama_french['Date'] <= end_date)]
momentum = momentum[(momentum['Date'] >= start_date) & (momentum['Date'] <= end_date)]

In [21]:
# Preprocessing
fama_french['Date'] = pd.to_datetime(fama_french['Date'], format='%Y%m%d')
momentum['Date'] = pd.to_datetime(momentum['Date'], format='%Y%m%d')
prices['Date'] = pd.to_datetime(prices['Date'])

# Calculate stock returns
chosen_stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']
stock_prices = prices[chosen_stocks]
stock_returns = np.log(stock_prices / stock_prices.shift(1))
stock_returns = stock_returns.dropna()
stock_returns['Date'] = prices['Date'][1:]

# Align the data
stock_returns = stock_returns.set_index('Date')
fama_french = fama_french.set_index('Date')
momentum = momentum.set_index('Date')

# Merge the stock returns, Fama-French factors, and momentum factors
merged_data = pd.concat([stock_returns, fama_french, momentum], axis=1).dropna()
stock_returns = merged_data[chosen_stocks]
fama_french = merged_data[['Mkt-RF', 'SMB', 'HML', 'RF']]
momentum = merged_data[['Mom']]

# Calculate excess returns
# fama_french[['Mkt-RF', 'SMB', 'HML', 'RF']] = fama_french[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100
# momentum['Mom'] = momentum['Mom']
# stock_returns.iloc[:, :-1] = stock_returns.iloc[:, :-1].sub(fama_french['RF'].values, axis=0)

In [22]:
# Merge the factor datasets
factors = fama_french.merge(momentum, on='Date', how='left')
factors = factors

# # Calculate excess stock returns
# stock_excess_returns = stock_returns.multiply(100) - factors['RF']

# # Fit the 4-factor model
# X = factors[['Mkt-RF', 'SMB', 'HML', 'Mom']]
# X = sm.add_constant(X)

# fitted_models = {}
# stocks = stock_returns.columns[:-1]
# for stock in stocks:
#     y = stock_excess_returns[stock]
#     model = sm.OLS(y, X).fit()
#     fitted_models[stock] = model

# # Calculate expected annual returns
# expected_returns = pd.Series(index=stocks)
# for stock, model in fitted_models.items():
#     annualized_alpha = (1 + model.params['const']) ** 252 - 1
#     expected_market_return = factors['Mkt-RF'].mean() * 252
#     annualized_expected_return = annualized_alpha + model.params['Mkt-RF'] * expected_market_return
#     expected_returns[stock] = annualized_expected_return

# # Calculate the annual covariance matrix
# cov_matrix = stock_excess_returns.cov() * 252

# Fit the 4-factor model to each stock
stocks = stock_returns.columns[:]
betas = []

for stock in stocks:
    reg = LinearRegression().fit(factors[['Mkt-RF', 'SMB', 'HML', 'Mom']], stock_returns[stock])
    betas.append(reg.coef_)

betas = pd.DataFrame(betas, index=stocks, columns=['Mkt-RF', 'SMB', 'HML', 'Mom'])

In [23]:
# Merge stock_returns, fama_french, and momentum data
combined_data = pd.merge(stock_returns, fama_french, on='Date')
combined_data = pd.merge(combined_data, momentum, on='Date')

# Calculate excess stock returns
excess_stock_returns = combined_data[chosen_stocks].subtract(combined_data['RF'], axis=0)

# Fit the 4-factor model to each stock
X = combined_data[['Mkt-RF', 'SMB', 'HML', 'Mom']]
y = excess_stock_returns

betas = []
intercepts = []

for stock in chosen_stocks:
    model = LinearRegression()
    model.fit(X, y[stock])
    betas.append(model.coef_)
    intercepts.append(model.intercept_)

# Estimate expected annual returns using alpha and beta
expected_excess_returns = np.array(intercepts) * 252 + np.dot(np.array(betas), np.array([combined_data['Mkt-RF'].mean(), combined_data['SMB'].mean(), combined_data['HML'].mean(), combined_data['Mom'].mean()])) * 252
expected_returns = expected_excess_returns + combined_data['RF'].mean() * 252
expected_returns = pd.Series(expected_returns, index=chosen_stocks)
expected_returns

AAPL    -0.293824
META    -0.675949
UNH      0.141270
MA      -0.078367
MSFT    -0.225500
NVDA    -0.577209
HD      -0.093769
PFE      0.060310
AMZN    -0.699768
BRK-B   -0.019689
PG      -0.016499
XOM      0.412650
TSLA    -0.983864
JPM     -0.117920
V       -0.085881
DIS     -0.629154
GOOGL   -0.489423
JNJ      0.103228
BAC     -0.383005
CSCO    -0.096899
dtype: float64

In [24]:
cov_matrix = stock_returns.iloc[:, :].cov() * 252
cov_matrix

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.132153,0.142101,0.043466,0.087146,0.108822,0.178193,0.070967,0.036699,0.130570,0.059993,0.040552,0.039497,0.155151,0.062709,0.076727,0.093862,0.115119,0.024762,0.070840,0.070864
META,0.142101,0.387574,0.035260,0.110263,0.140498,0.242295,0.097602,0.048733,0.189288,0.064774,0.038304,0.028357,0.172741,0.079433,0.091430,0.123813,0.170535,0.020531,0.091446,0.076820
UNH,0.043466,0.035260,0.059603,0.034765,0.040331,0.055192,0.031235,0.033841,0.044192,0.031248,0.030523,0.025869,0.043960,0.037053,0.033366,0.028829,0.037653,0.025642,0.039811,0.032419
MA,0.087146,0.110263,0.034765,0.101981,0.087006,0.146206,0.061001,0.036800,0.103929,0.050422,0.032927,0.033688,0.104786,0.063408,0.089150,0.082487,0.084404,0.018306,0.069046,0.054920
MSFT,0.108822,0.140498,0.040331,0.087006,0.128227,0.180669,0.073888,0.037519,0.135456,0.056225,0.037445,0.032903,0.133689,0.061470,0.075013,0.092326,0.119668,0.021683,0.069806,0.063596
NVDA,0.178193,0.242295,0.055192,0.146206,0.180669,0.408518,0.117225,0.056328,0.229966,0.090600,0.046195,0.059969,0.284236,0.105525,0.125628,0.161962,0.192712,0.022772,0.120691,0.103885
HD,0.070967,0.097602,0.031235,0.061001,0.073888,0.117225,0.101662,0.035888,0.097091,0.044489,0.036526,0.018861,0.081913,0.047021,0.054670,0.067642,0.070348,0.023456,0.049223,0.050804
PFE,0.036699,0.048733,0.033841,0.036800,0.037519,0.056328,0.035888,0.070870,0.040251,0.033021,0.028231,0.020949,0.033913,0.033864,0.034372,0.027841,0.031884,0.027821,0.032468,0.030850
AMZN,0.130570,0.189288,0.044192,0.103929,0.135456,0.229966,0.097091,0.040251,0.243964,0.069379,0.033156,0.044143,0.196843,0.077063,0.090484,0.128901,0.147191,0.023772,0.090619,0.073005
BRK-B,0.059993,0.064774,0.031248,0.050422,0.056225,0.090600,0.044489,0.033021,0.069379,0.053136,0.026053,0.036046,0.068003,0.050445,0.045320,0.054775,0.058494,0.019910,0.054339,0.042671


In [25]:
rf = 0.0425

def neg_sharpe_ratio(weights, expected_returns, cov_matrix, rf):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - rf) / portfolio_volatility
    return -sharpe_ratio

# Optimization constraints
num_stocks = len(chosen_stocks)
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
bounds = [(0, 1) for _ in range(num_stocks)]

initial_weights = np.repeat(1/num_stocks, num_stocks)

# Find the optimal weights for the super efficient portfolio
result = minimize(neg_sharpe_ratio, initial_weights, args=(expected_returns, cov_matrix, rf), bounds=bounds, constraints=constraints)
optimal_weights = result.x

super_efficient_portfolio = pd.DataFrame(optimal_weights, index=chosen_stocks, columns=['Weight']).round(4)
super_efficient_portfolio

,Weight
AAPL,0.0000
META,0.0000
UNH,0.0000
MA,0.0000
MSFT,0.0000
NVDA,0.0000
HD,0.0000
PFE,0.0000
AMZN,0.0000
BRK-B,0.0000


In [26]:
result

     fun: -1.070565967959299
     jac: array([ 1.73978718,  3.09900674, -0.12737295,  0.78910802,  1.37890676,
        3.07992177,  0.70911752,  0.14795443,  3.38940388,  0.59281389,
        0.24531688, -0.16748096,  4.43860234,  0.91794457,  0.72767271,
        3.01300728,  2.42510642, -0.16828734,  2.00676511,  0.79262966])
 message: 'Optimization terminated successfully'
    nfev: 169
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([0.00000000e+00, 0.00000000e+00, 5.75298811e-14, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.83719123e-15, 1.42143830e-14,
       0.00000000e+00, 4.12852095e-15, 1.90545388e-14, 6.80369314e-01,
       0.00000000e+00, 1.91008829e-16, 1.41415497e-15, 0.00000000e+00,
       0.00000000e+00, 3.19630686e-01, 0.00000000e+00, 3.15357454e-15])